# NLTK

Fist, install some packs

```
pip3 install -U spacy  
```

```
python3 -m spacy download en
```

```
pip install pyldavis
```

ref: https://datascienceplus.com/topic-modeling-in-python-with-nltk-and-gensim/

In [4]:
import spacy
spacy.load('en')
from spacy.lang.en import English
parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [5]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

[nltk_data] Downloading package wordnet to /Users/zireael/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [16]:
# nltk.download('stopwords')
# en_stop = set(nltk.corpus.stopwords.words('english'))

from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))


new_stopwords = ['|'] #no neet since the file was updated. more info to came
new_stopwords_list = stop_words.union(new_stopwords)

print(new_stopwords_list)

{'had', 'being', 'nor', 'into', 'don', 'do', 'the', 'myself', "doesn't", 'down', "don't", 'through', "weren't", 'a', 'here', 'an', 'same', 'below', "won't", "couldn't", 'both', 'doing', "you'll", 'this', 'hadn', 'but', 'there', "you'd", 'is', "shan't", 'himself', 'ours', "didn't", 'can', 'each', 'just', 'no', "needn't", 'in', 're', 'wasn', 'again', 'shan', 'as', 'any', "hasn't", 'me', 'have', 'been', "isn't", 'between', 'further', '|', "aren't", 'mustn', 'my', 'will', 'or', 'ain', 'to', 'are', 'hasn', 'shouldn', 'from', 'more', 'she', 'over', 'other', 'very', 'ourselves', 'yourselves', 'once', 'm', 'has', 'he', 'such', 'theirs', 'during', 'yourself', 'themselves', 'y', 'ma', 'we', "mustn't", 'did', 'so', 'those', 'you', 'herself', 'does', 'isn', 'before', 'mightn', 'haven', 'above', "wouldn't", 'him', 'up', 've', "that'll", 'they', 'were', 'against', 'i', "you're", 'won', 'your', 'then', 'only', 'hers', 'when', 'for', 'couldn', 'not', 'by', "it's", 'aren', 'needn', 'd', 'didn', 'these'

In [17]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in new_stopwords]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

Topic model file is the database only replacing some values so it will have a aproximate behaviour as a textual document, as having space between values(representing words) and replacing every space with a underscore so the 'palavara composta?' dont be lost without its context

In [37]:
import random
text_data = []

with open('data/treated_data/topic-model.csv') as f:
    for line in f:
        tokens = prepare_text_for_lda(line)
        if random.random() > .99:
            print(tokens)
            text_data.append(tokens)

['56359', '1,1,1,3,3,3-hexabutyldistannoxane', 'not_reported', 'mammal', 'mus_musculus', 'house_mouse', 'not_reported', 'not_reported', 'no_substrate', 'gavage', 'mortality', 'mortality', 'not_reported', 'unmeasured', 'total', 'day(s', 'davis', 'a.,_r._barale,_g._brun,_r._forster,_t._gunther,_h._hautefeuille,_c.a._van_der_heijden,_a.g.a.c._knaap,_r._krowke', '61037', 'evaluation_of_the_genetic_and_embryotoxic_effects_of_bis(tri', 'butyltin)_oxide_(tbto),_a_broad', 'spectrum_pesticide,_in_multiple_in_vivo_and_in_vitro_short', 'term_tests', 'mutat._res.188:65']
['58366', "10,10'-oxybis-10h", 'phenoxarsine', 'not_reported', 'mammal', 'cavia_porcellus', 'guinea_pig', 'not_reported', 'not_reported', 'no_substrate', 'gavage', 'mortality', 'mortality', 'not_reported', 'unmeasured', 'total', 'day(s', 'ballantyne', '35725', 'the_comparative_short', 'term_mammalian_toxicology_of_phenarsazine_oxide_and_phenoxarsine_oxide', 'toxicology10:341']
['60571', '1ar,2r,2as,3s,6r,6ar,7s,7as)-rel-3,4,5,6,9,

In [43]:
from gensim import corpora

dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [44]:
import gensim
NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.007*"not_reported" + 0.007*"mortality" + 0.007*"not_coded" + 0.007*"intraperitoneal"')
(1, '0.184*"not_reported" + 0.079*"mortality" + 0.040*"day(s" + 0.031*"mammal"')
(2, '0.007*"not_reported" + 0.007*"mortality" + 0.007*"not_coded" + 0.007*"intraperitoneal"')
(3, '0.007*"not_reported" + 0.007*"mortality" + 0.007*"not_coded" + 0.007*"intraperitoneal"')
(4, '0.098*"not_reported" + 0.078*"mortality" + 0.066*"not_coded" + 0.046*"day(s"')


In [45]:
new_doc = 'Practical Bayesian Optimization of Machine Learning Algorithms'
new_doc = prepare_text_for_lda(new_doc)
new_doc_bow = dictionary.doc2bow(new_doc)
print(new_doc_bow)
print(ldamodel.get_document_topics(new_doc_bow))

[]
[(0, 0.2), (1, 0.2), (2, 0.2), (3, 0.2), (4, 0.2)]


In [47]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')

import pyLDAvis.gensim

lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

/usr/local/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
